# Energiegleichung (2D)

## Einführung

In der letzten Übung haben wir die Advektion für ein eindimensionales Problem betrachtet. Nun wollen wir die Advektion in zwei Dimensionen betrachten. 

Bisher haben wir zudem Advektion und Diffusion als zwei seperate Prozesse betrachtet und verschiedene numerische Verfahren zur Lösung des jeweiligen Prozesses verwendet. Allerdings sind wir bei vielen Problemen der Geowissenschaften an dem Transport und der Diffusion gleichzeitig interessiert. Die Energieerhaltungsgleichung ist in diesem Fall gegeben durch (Annahme: konstante thermische Parameter):

$\begin{equation}
\frac{\partial{T}}{\partial{t}}=-v_x\frac{\partial{T}}{\partial{x}} - v_y\frac{\partial{T}}{\partial{y}} + \kappa\left(\frac{\partial^2{T}}{\partial{x^2}} + \frac{\partial^2{T}}{\partial{y^2}}\right), 
\end{equation}$

wobei $T$ die Temperatur [K], $t$ die Zeit [s], $v_i$ die Geschwindigkeit in Raumrichtung $i$ [m/s], und $\kappa = k/\rho/c_p$ die thermische Diffusivität [m²/s]. 

Die ersten beiden Terme der rechten Seite der Gleichung beschreiben die **Advektion**, der mittlere Term die **Diffusion**, und der letzte Term die **Wärmeproduktionsrate**. Alle drei Term beschreiben die Änderung der Temperatur mit der Zeit $\frac{\partial{T}}{\partial{t}}$ an einem bestimmten Ort. 

Hier wollen wir die vordefinierten Funktionen für Advektion und Diffusion aus den Submodulen `GeoModBox.AdvectionEquation.TwoD` und `GeoModBox.HeatEquation.TwoD` verwenden. Für mehr Informationen über die Diskretisierung und Implementierung der Advektionsgleichung im zwei dimensionalen seht bitte die [Dokumentation](https://geosci-ffm.github.io/GeoModBox.jl/dev/man/AdvTwoD/).## Einführung

## Das Problem

Im folgenden betrachten wir, wie sich das Temperaturfeld in einem konstantem, zweidimensionalem Geschwindigkeitsfeld und durch diffusive Prozesse mit der Zeit verändert. Als Anfangsbedingungen sind jeweils unterschiedliche Bedingungen definierbar. 

Für die Temperatur: 

1) Eine kreisförmige Anomalie mit konstanter Hintergrundtemperatur
2) Eine Gaussche Temperaturverteilung
3) Eine rechteckige Anomalie mit konstanter Hintergrundtemperatur
4) Eine linear ansteigende Temperatur

Für die Geschwindigkeit: 

1) Eine starre Rotation
2) Eine analytisch beschriebene Konvektionszelle

Die Anfangsposition der Anomalie ist wie in Abbildung 1 dargestellt und im Falle der starre Rotation werden die Geschwindigkeiten außerhalb des Rotationsradius auf null gesetzt. Dies verhindert, dass Partikel nicht aus der Modeldomaine advektiert werden. 

<img src="./Figures/Exercise07_1.png" al7t="drawing" width="500"/> <br>

**Abbildung 1.** Setup für eine starre Rotation mit einer Temperaturanomolie.  

## Die Lösung

<img src="./Figures/Exercise07_grid.png" al7t="drawing" width="500"/> <br>

**Abbildung 2.** Numerisches Gitter zur Lösung des Problems. 

Es gibt mehrere Verfahren mit denen man die Energieerhaltung numerisch lösen kann (die Art und Weise der Diskretisierung der Gleichung ist dabei eintscheidend). 

Hier wollen wir uns auf ein einfaches Verfahren konzentrieren, das Verfahren der sogenannten Operatoren-Teilung(„*operator-splitting*“).

In erster Näherung lässt sich so die Energiegleichung durch die Trennung und das separate Lösen der Gleichungen lösen. Das heißt, in der Zeitschleife wird zuerst die Advektionsgleichung gelöst (z.B. durch das Semi-Lagrange Verfahren): 

$\begin{equation}
\frac{\tilde{T}^{n+1}-T^n}{\Delta{t}} = -v_x\frac{\partial{T}}{\partial{x}}-v_y\frac{\partial{T}}{\partial{y}},
\end{equation}$

gefolgt von der Lösung der Diffusionsgleichung (z.B. mit dem Crank-Nicholson Verfahren): 

$\begin{equation}
\frac{\partial{\tilde{T}}}{\partial{t}} = \kappa\left(\frac{\partial^2{\tilde{T}}}{\partial{x^2}}+\frac{\partial^2{\tilde{T}}}{\partial{y^2}}\right).
\end{equation}$

Beachtet dabei die Annahmen, die für die Gleichungen gelten. Für mehr Informationen über die Diskretisierung und Implementierung der Advektionsgleichung und der Diffusionsgleichung im zwei dimensionalen seht bitte die entsprechende [Dokumentation](https://geosci-ffm.github.io/GeoModBox.jl/). 

Im folgenden wollen wir die vordefinierten Funktionen für die Lösung der Advektion- und Diffusionsgleichung aus den Submodulen `GeoModBox.AdvectionEquation.TwoD` und `GeoModBox.HeatEquation.TwoD` verwenden. Dazu müssen wir wie immer zuerst die notwendigen Module laden: 

In [ ]:
using Plots, Interpolations, ExtendableSparse, LinearAlgebra
using GeoModBox
using GeoModBox.AdvectionEquation.TwoD
using GeoModBox.HeatEquation.TwoD
using GeoModBox.InitialCondition
using GeoModBox.Tracers.TwoD # LuFu
using Base.Threads, Printf

In [ ]:
# Definition numerischer Verfahren =================================== #
# Define Advection Scheme ---
#   1) upwind, 2) slf, 3) semilag, 4) tracers
# Define Diffusion Scheme --- 
#   1) explicit, 2) implicit, 3) CNA, 4) ADI, 5) dc
FD          =   (Method     = (Adv=:upwind,Diff=:ADI),)
# Define Initial Condition ---
# Temperature - 
#   1) circle, 2) gaussian, 3) block
# Velocity - 
#   1) RigidBody, 2) ShearCell
Ini         =   (T=:circle,V=:RigidBody,) 
# -------------------------------------------------------------------- #

Die folgenden Parameter definieren die Eigenschaften der Visualisierungen: 

In [ ]:
# Plot Einstellungen ================================================= #
Pl  =   (
    inc         =   5,
    sc          =   1.0*(100*(60*60*24*365.15)),        
    Minc        =   1, 
    Msz         =   0.2,
)
# Animationssettings ---
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("07_2D_EnergyEquation_",Ini.T,"_",Ini.V,
                        "_",FD.Method.Adv,"_",FD.Method.Diff)
save_fig    =   0
# -------------------------------------------------------------------- #

Als nächsten definieren wir die Modelgeometrie und die physikalischen Konstanten: 

In [ ]:
# Model Konstanten =================================================== #
M   =   (
    xmin    =    0.0,
    xmax    =   100.0e3,
    ymin    =    0.0,
    ymax    =   100.0e3,
)
# -------------------------------------------------------------------- #
# Physical Parameters ================================================ #
P       = ( 
    k       =   3,              #   Thermal Conductivity [ W/m/K ]
    cp      =   1000,           #   Specific Heat Capacity [ J/kg/K ]
    ρ       =   3200,           #   Density [ kg/m^3 ]
    K0      =   273.15,         #   Kelvin at 0 °C
    Q0      =   0,               #   Heat production rate
)
P1      = (
    κ       =    P.k/P.cp/P.ρ,   #   Thermal Diffusivity [ m^2/s ] 
)
P       =   merge(P,P1)
# -------------------------------------------------------------------- #

Nun folgt die Definition der numerischen Parameter ...

In [ ]:
# Numerische Konstanten ============================================== #
NC  =   (
    x       =   100,        # Number of horizontal centroids
    y       =   100,        # Number of vertical centroids
)
NV  =   (
    x       =   NC.x + 1,  # Number of horizontal vertices
    y       =   NC.y + 1,  # Number of vertical vertices
)
Δ   =   (
    x   =   (M.xmax - M.xmin) / NC.x,
    y   =   (M.ymax - M.ymin) / NC.y,
)
# -------------------------------------------------------------------- #

... und der Gitterkoordinaten.

In [ ]:
# Erstellung des Gitters ============================================= #
x   =   (
    c       =   LinRange(M.xmin + Δ.x/2, M.xmax - Δ.x/2, NC.x),
    ce      =   LinRange(M.xmin - Δ.x/2, M.xmax + Δ.x/2, NC.x + 2),
    v       =   LinRange(M.xmin, M.xmax, NC.x + 1)
)
y       = (
    c       =   LinRange(M.ymin + Δ.y/2, M.ymax - Δ.y/2, NC.y),
    ce      =   LinRange(M.ymin - Δ.y/2, M.ymax + Δ.y/2, NC.y + 2),
    v       =   LinRange(M.ymin, M.ymax, NC.y + 1)
)
x1      =   ( 
    c2d     =   x.c .+ 0*y.c',
    v2d     =   x.v .+ 0*y.v', 
    vx2d    =   x.v .+ 0*y.ce',
    vy2d    =   x.ce .+ 0*y.v',
)
x   =   merge(x,x1)
y1      =   (
    c2d     =   0*x.c .+ y.c',
    v2d     =   0*x.v .+ y.v',
    vx2d    =   0*x.v .+ y.ce',
    vy2d    =   0*x.ce .+ y.v',
)
y   =   merge(y,y1)
# -------------------------------------------------------------------- #

Hier werden mit Hilfe der Strukturen der GeoModBox die Datenfelder definiert. 

In [ ]:
# Felder Initialisierung ============================================= #
D       =   DataFields(
    Q       =   zeros(Float64,(NC.x,NC.y)),
    T       =   zeros(Float64,(NC.x,NC.y)),
    T0      =   zeros(Float64,(NC.x,NC.y)),
    T_ex    =   zeros(Float64,(NC.x+2,NC.y+2)),
    T_exo   =   zeros(Float64,(NC.x+2,NC.y+2)),
    ρ       =   zeros(Float64,(NC.x,NC.y)),
    cp      =   zeros(Float64,(NC.x,NC.y)),
    vx      =   zeros(Float64,(NV.x,NV.y+1)),
    vy      =   zeros(Float64,(NV.x+1,NV.y)),    
    vxc     =   zeros(Float64,(NC.x,NC.y)),
    vyc     =   zeros(Float64,(NC.x,NC.y)),
    vc      =   zeros(Float64,(NC.x,NC.y)),
    wt      =   zeros(Float64,(NC.x,NC.y)),
    wte     =   zeros(Float64,(NC.x+2,NC.y+2)),
    wtv     =   zeros(Float64,(NV...)),
    Tmax    =   0.0,
    Tmin    =   0.0,
    Tmean   =   0.0,
)
# -------------------------------------------------------------------- #

Jetz können wir mit Hilfe der vordefinierten Funktionen die Anfangstemperatur- und Geschwindigkeitsfelder berechnen. 

In [ ]:
# Anfangsbedingungen ================================================= #
# Temperatur ---
IniTemperature!(Ini.T,M,NC,D,x,y;Tb=250.0,Ta=1200.0,σ=0.1)
D.Tmax  =   maximum(D.T_ex)                    # Maximum temperature
D.Tmin  =   minimum(D.T_ex)                    # Minimum temperature
D.Tmean =   (D.Tmax[1] + D.Tmin[1]) / 2        # Mean temperature
if FD.Method.Adv==:slf
    D.T_exo    .=  D.T_ex
end
# Heat production rate ---
@. D.Q  =   P.Q0
@. D.ρ  =   P.ρ
# Velocity ---
# RBR - maximum(v) = 0.5 cm/a
IniVelocity!(Ini.V,D,0,NV,Δ,M,x,y)        # [ m/s ]
# Upscale velocity ---
D.vx    .*=     20.0
D.vy    .*=     20.0
# Get the velocity on the centroids ---
@threads for i = 1:NC.x
    for j = 1:NC.y
        D.vxc[i,j]  =  0.5 * (D.vx[i,j] + D.vx[i+1,j])
        D.vyc[i,j]  =  0.5 * (D.vy[i,j] + D.vy[i,j+1])
    end
end
@. D.vc        = sqrt(D.vxc^2 + D.vyc^2)
@show minimum(D.vc),maximum(D.vc)
# -------------------------------------------------------------------- #

Da die Geschwindigkeitsfelder konstant und vorgeschrieben sind, müssen nur die Randbedingungen für die Temperatur definiert werden. 

In [ ]:
# Boundary Conditions ================================================ #
# BC     = (type    = (W=:Dirichlet, E=:Dirichlet, 
#                     N=:Dirichlet, S=:Dirichlet),
#         val     = (W=250.0,E=250.0,
#                     N=250.0,S=250.0))
BC     = (type    = (W=:Dirichlet, E=:Dirichlet, 
                    N=:Dirichlet, S=:Dirichlet),
        val     = (W=D.T[1,:],E=D.T[end,:],
                    N=D.T[:,end],S=D.T[:,1]))
# ------------------------------------------------
# -------------------------------------------------------------------- #

Abhängig von der numerischen Methode zur Lösung der Diffusionsgleichung müssen bestimmte Parameter definiert werden, wie zum Beispiel die Koeffizienten Matrizen und die rechte Seite der Gleichungssysteme oder Parameter für eine Iteration. Für mehr Informationen bezüglich der Lösungsverfahren der Diffusionsgleichung im zweidimensionalem, siehe die [Dokumentation](https://geosci-ffm.github.io/GeoModBox.jl/dev/man/DiffTwoD/)

In [ ]:
# Linear systems ======================================================= #
if FD.Method.Diff == :implicit || FD.Method.Diff == :CNA
    # Numbering of unknowns (flattened 2D grid into 1D index)
    Num     =   (T = reshape(1:NC.x*NC.y, NC.x, NC.y),)
    ndof    =   maximum(Num.T)                     # Degrees of freedom
    
    if FD.Method.Diff == :CNA
        # Two coefficient matrices for Crank–Nicolson Approach
        K1      =   ExtendableSparseMatrix(ndof, ndof)
        K2      =   ExtendableSparseMatrix(ndof, ndof)
    else
        # Single coefficient matrix for implicit scheme
        K       =   ExtendableSparseMatrix(ndof, ndof)
    end
    rhs     =   zeros(ndof)                        # Right-hand side vector

elseif FD.Method.Diff == :dc
    # Defect-correction iteration parameters
    niter       =   10                             # Max iterations
    ϵ           =   1e-10                          # Convergence tolerance
    
    # Physical fields
    @. D.ρ      =   P.ρ
    @. D.cp     =   P.cp
    
    # Thermal conductivity on cell faces
    k           =   (x = zeros(NC.x+1, NC.y),
                     y = zeros(NC.x,   NC.y+1))
    @. k.x      =   P.k
    @. k.y      =   P.k
    
    # Numbering of unknowns
    Num         =   (T = reshape(1:NC.x*NC.y, NC.x, NC.y),)
    ndof        =   maximum(Num.T)
    
    # System matrix and residual
    K           =   ExtendableSparseMatrix(ndof, ndof)
    R           =   zeros(NC.x, NC.y)              # Residual field
    
    # Temperature gradients at faces
    ∂T          =   (∂x = zeros(NC.x+1, NC.y),
                     ∂y = zeros(NC.x,   NC.y+1))
    
    # Heat fluxes at faces
    q           =   (x  = zeros(NC.x+1, NC.y),
                     y  = zeros(NC.x,   NC.y+1),
                     x0  = zeros(NC.x+1, NC.y),
                     y0  = zeros(NC.x,   NC.y+1))
end
# ---------------------------------------------------------------------- #


Nun können wir die Zeit Parameter bestimmen. Beachte: nun muss der größte mögliche Zeitschritt für das Advektion- und Diffusionverfahren berücksichtigt werden. 

In [ ]:
# Time ================================================================ #
T   =   ( 
    tmax    =   [0.0],     # Maximum simulation time [s]
    Δfacc   =   1.0,       # Courant time factor, i.e., dtfac * dt_courant
    Δfacd   =   1.0,       # Diffusion time factor, i.e., dtfac * dt_diff 
    Δ       =   [0.0],     # Final chosen time step
    Δc      =   [0.0],     # Courant time step (advection stability limit)
    Δd      =   [0.0],     # Diffusion time step (stability criterion)
)

# Maximum runtime: based on a characteristic advection time scale
# here approximated by π * (domain width – one cell) / maximum velocity
T.tmax[1]   =   π * ((M.xmax - M.xmin) - Δ.x) / maximum(D.vc)   # [s]

# Courant stability condition (for advection)
# T.Δc[1]     =   (( Δ.x / minimum(D.vx)) + (Δ.y / minimum(D.vy))) *  T.Δfacc
T.Δc[1]     =   T.Δfacc * minimum((Δ.x,Δ.y)) / 
                    (sqrt(maximum(abs.(D.vx))^2 + maximum(abs.(D.vy))^2))

# Diffusion stability condition (explicit scheme)
# T.Δd[1]     =   (1 /(4 * P.κ)) * ((Δ.x^2 + Δ.y^2)/2)* T.Δfacd
T.Δd[1]     =   T.Δfacd * (1.0 / (2.0 * P.κ *(1.0/Δ.x^2 + 1/Δ.y^2))) # LuFu

steps = [T.Δc[1] , T.Δd[1]]

# This is the final time step length! 
# Final time step: take the most restrictive (advection or diffusion)
# T.Δ[1]      =   maximum(steps)
T.Δ[1]      =   minimum([T.Δd[1],T.Δc[1]]) # LuFu

@show T.Δ[1], T.Δd[1], T.Δc[1]

# Total number of time steps
nt          =   ceil(Int, T.tmax[1] / T.Δ[1])
# --------------------------------------------------------------------- #

Für den Fall das die Temperatur mit Tracern advektiert werden soll, werden im nächsten Schritt die Tracer initialisiert. 

In [ ]:
# Tracer Advection =================================================== #
if FD.Method.Adv==:tracers 
    # Tracer Initialization ---
    nmx,nmy     =   3,3
    noise       =   1
    nmark       =   nmx*nmy*NC.x*NC.y
    Aparam      =   :thermal
    MPC         =   (
        c       =   zeros(Float64,(NC.x,NC.y)),
        v       =   zeros(Float64,(NV.x,NV.y)),
        th      =   zeros(Float64,(nthreads(),NC.x,NC.y)),
        thv     =   zeros(Float64,(nthreads(),NV.x,NV.y)),
    )
    MAVG        = (
        PC_th   =   [similar(D.wte) for _ = 1:nthreads()],  # per thread
        PV_th   =   [similar(D.wtv) for _ = 1:nthreads()],   # per thread
        wte_th  =   [similar(D.wte) for _ = 1:nthreads()],  # per thread
        wtv_th  =   [similar(D.wtv) for _ = 1:nthreads()],  # per thread
    )
    Ma      =   IniTracer2D(Aparam,nmx,nmy,Δ,M,NC,noise,0,0)
    # RK4 weights ---
    rkw     =   1.0/6.0*[1.0 2.0 2.0 1.0]   # for averaging
    rkv     =   1.0/2.0*[1.0 1.0 2.0 2.0]   # for time stepping
    # Interpolate on centroids ---
    @threads for k = 1:nmark
        Ma.T[k] =   FromCtoM(D.T_ex, k, Ma, x, y, Δ, NC)
    end
    # Count marker per cell ---
    CountMPC(Ma,nmark,MPC,M,x,y,Δ,NC,NV)
end
# -------------------------------------------------------------------- #

Lasst uns nun die Anfangsbedingungen plotten.

In [ ]:
# Visualize initial condition ======================================== #
p = heatmap(x.c./1e3 , y.c./1e3, (D.T./D.Tmax[1])', 
    color=:thermal, colorbar=true, aspect_ratio=:equal, 
    xlabel="x [km]", ylabel="z[km]", 
    title="Temperature", 
    xlims=(M.xmin./1e3, M.xmax./1e3), ylims=(M.ymin./1e3, M.ymax./1e3),
    clim=(0.5, 1.0))
quiver!(p,x.c2d[1:Pl.inc:end,1:Pl.inc:end]./1e3,
        y.c2d[1:Pl.inc:end,1:Pl.inc:end]./1e3,
        quiver=(D.vxc[1:Pl.inc:end,1:Pl.inc:end].*Pl.sc,
            D.vyc[1:Pl.inc:end,1:Pl.inc:end].*Pl.sc),        
        color="white")
if save_fig == 1
    Plots.frame(anim)
elseif save_fig == 0
    display(p)
end
# -------------------------------------------------------------------- #

Nun können wir die Zeitschleift starten. Innerhalb der Zeitschleife müssen wir zuerst die Advektiongleichung, gefolgt von der Diffusionsgleichung lösen. Anschließend wir das Temperaturfeld für bestimmte Zeitschritte geplottet. 

In [ ]:
# Zeitschleife ======================================================= #
for i=2:nt
    @printf("Time step: #%04d\n ",i)
    
    # Loesung Advektionsgleichung ---
    if FD.Method.Adv==:upwind
        upwindc2D!(D.T,D.T_ex,D.vxc,D.vyc,NC,T.Δ[1],Δ.x,Δ.y)
    elseif FD.Method.Adv==:slf
        # SLFc2D!(D.T,D.T_ex,D.vxc,D.vyc,NC,T.Δ[1],Δ.x,Δ.y)    
        slfc2D!(D.T,D.T_ex,D.T_exo,D.vxc,D.vyc,NC,T.Δ[1],Δ.x,Δ.y) # LuFu
    elseif FD.Method.Adv==:semilag
    #   semilagc2D!(D.T,D.T_ex,D.vxc,D.vyc,D.vxo,D.vyo,x,y,T.Δ[1])   
        semilagc2D!(D.T,D.T_ex,D.vxc,D.vyc,[],[],x,y,T.Δ[1])
    elseif FD.Method.Adv==:tracers
        # NEED TO GET RID OF THE NAN'S IN D.T FIRST!
        ## Interpolate from grid on tracers ---
        @threads for k = 1:nmark
           Ma.T[k] =   FromCtoM(D.T_ex, k, Ma, x, y, Δ, NC)
        end
        # Advect tracers ---
        AdvectTracer2D(Ma,nmark,D,x,y,T.Δ[1],Δ,NC,rkw,rkv;style=1)
        # AdvectTracer2D(Ma,nmark,D,x,y,T.Δ[1],Δ,NC,rkw,rkv,1)
        # CountMPC(Ma,nmark,MPC,M,x,y,Δ,NC,NV,i)
        CountMPC(Ma,nmark,MPC,M,x,y,Δ,NC,NV)
        
        # Interpolate temperature from tracers to grid ---
        Markers2Cells(Ma,nmark,MAVG.PC_th,D.T_ex,MAVG.wte_th,D.wte,x,y,Δ,Aparam,0)           
        D.T     .=  D.T_ex[2:end-1,2:end-1]
    end

    # Loesung Diffusionsgleichung ---
    if FD.Method.Diff==:explicit
        # @show size(D.T_ex), size(D.Q), size(D.ρ), size(D.cp)
        ForwardEuler2Dc!(D, P.κ, Δ.x, Δ.y, T.Δ[1], D.ρ, P.cp, NC, BC)
    elseif FD.Method.Diff==:CNA
        # @show BC
        CNA2Dc!(D, P.κ, Δ.x, Δ.y, T.Δ[1], D.ρ, P.cp, NC, BC, rhs, K1, K2, Num)
    elseif FD.Method.Diff==:ADI
        ADI2Dc!(D, P.κ, Δ.x, Δ.y, T.Δ[1], D.ρ, P.cp, NC, BC)
    elseif FD.Method.Diff==:dc
        D.T0    .=  D.T
        for iter = 1:niter
            # Evaluate residual
            # ComputeResiduals2D!(R, D.T, D.T_ex, D.T0, ∂T, q, D.ρ, D.cp, k, BC, Δ, T.Δ[1])
            ComputeResiduals2D!(R, D.T, D.T_ex, D.T0, D.T_exo, D.Q, ∂T, 
                            q, D.ρ, D.cp, k, BC, Δ, T.Δ[1])
            # @printf("||R|| = %1.4e\n", norm(R)/length(R))
            norm(R)/length(R) < ϵ ? break : nothing
            # Assemble linear system
            # K  = AssembleMatrix2D(D.ρ, D.cp, k, BC, Num, NC, Δ, T.Δ[1])
            K  = AssembleMatrix2D(D.ρ, D.cp, k, BC, Num, NC, Δ, T.Δ[1])
            # Solve for temperature correction: Cholesky factorisation
            Kc = cholesky(K.cscmatrix)
            # Solve for temperature correction: Back substitutions
            δT = -(Kc\R[:])
            # Update temperature
            @. D.T += δT[Num.T]
        end
    end

    @printf("ΔT = %4.4f\n\n",abs((D.Tmax[1]-maximum(filter(!isnan,D.T)))/D.Tmax[1]*100))

    # Plot Solution ---
    if mod(i,10) == 0 || i == nt
        p = heatmap(x.c./1e3 , y.c./1e3, (D.T./D.Tmax[1])', 
            color=:thermal, colorbar=true, aspect_ratio=:equal, 
            xlabel="x [km]", ylabel="z[km]", 
            title="Temperature", 
            xlims=(M.xmin./1e3, M.xmax./1e3), ylims=(M.ymin./1e3, M.ymax./1e3),
            clim=(0.5, 1.0))
        quiver!(p,x.c2d[1:Pl.inc:end,1:Pl.inc:end]./1e3,
                y.c2d[1:Pl.inc:end,1:Pl.inc:end]./1e3,
                quiver=(D.vxc[1:Pl.inc:end,1:Pl.inc:end].*Pl.sc,
                        D.vyc[1:Pl.inc:end,1:Pl.inc:end].*Pl.sc),        
            color="white")
        if save_fig == 1
            Plots.frame(anim)
        elseif save_fig == 0
            display(p)                        
        end
    end

end # End Time loop ---
# -------------------------------------------------------------------- #

Nun wollen wir noch die Animation speichern. 

In [ ]:
# Save Animation ===================================================== #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
    foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
elseif save_fig == 0
    display(plot(p))
end
# -------------------------------------------------------------------- #